In [31]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import LSTM





import sklearn
from sklearn import preprocessing, model_selection
import tensorflow as tf

import src.lstm_helper as helper

In [139]:
#reference:https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
# https://heartbeat.fritz.ai/building-a-neural-network-from-scratch-using-python-part-1-6d399df8d432
# https://www.kaggle.com/alexdance/store-item-combination-part-6-deep-learning
# https://www.tensorflow.org/tutorials/structured_data/time_series

### Predict future sales but non time-series models.

#### Fearture engineering

In [5]:
path = 'data/processed_train.csv'
df = pd.read_csv(path)
df.shape

(913000, 13)

In [6]:
df.head()

,date,store,item,sales,year,month,day,day_of_week,year_and_month,weekend,quarter,week_block_num,quarter_block_num
0,2013-01-01,1,1,13,2013,1,1,1,2013-01,0,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2013-01,0,1,1,1
2,2013-01-03,1,1,14,2013,1,3,3,2013-01,0,1,1,1
3,2013-01-04,1,1,13,2013,1,4,4,2013-01,0,1,1,1
4,2013-01-05,1,1,10,2013,1,5,5,2013-01,1,1,1,1


In [7]:
#proceese data
processed_df = helper.data_pipeline(path)

In [8]:
processed_df.shape

(913000, 69)

In [9]:
processed_df.head()

,date,store,item,sales,month,day,day_of_week,weekend,quarter,week_block_num,...,i_41,i_42,i_43,i_44,i_45,i_46,i_47,i_48,i_49,i_50
0,2013-01-01,1,1,13,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2013-01-02,1,1,11,1,2,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2013-01-03,1,1,14,1,3,3,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2013-01-04,1,1,13,1,4,4,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2013-01-05,1,1,10,1,5,5,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#drop unique store & item Id
drop_cols =['store','item']

In [11]:
processed_df=processed_df.drop(columns=drop_cols,axis =1)

In [12]:
#set datatime to index
processed_df['date'] = pd.to_datetime(processed_df['date'])
processed_df = processed_df.set_index('date')
processed_df.head()

,sales,month,day,day_of_week,weekend,quarter,week_block_num,quarter_block_num,s_2,s_3,...,i_41,i_42,i_43,i_44,i_45,i_46,i_47,i_48,i_49,i_50
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,13,1,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-01-02,11,1,2,2,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-01-03,14,1,3,3,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-01-04,13,1,4,4,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-01-05,10,1,5,5,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#split into train and valid sets
train = processed_df.loc['2013-01-01':'2017-09-30']
valid = processed_df.loc['2017-10-01':'2017-12-31']

In [14]:
y_train = train['sales'].values.reshape(-1,1)
X_train = train.drop('sales',axis=1)
y_test  = valid['sales'].values.reshape(-1,1)
X_test = valid.drop('sales',axis=1)

In [15]:
print(y_train.shape)
print(X_train.shape)
print(y_test.shape)
print(X_test.shape)

(867000, 1)
(867000, 65)
(46000, 1)
(46000, 65)


In [16]:
processed_df.sales.sum() == (sum(y_test)+sum(y_train))

array([ True])

### Normalize

#### Normalize the data

In [17]:
#standardize the dataset
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)


In [18]:
print(f"Shape of train set is {X_train.shape}")
print(f"Shape of test set is {X_test.shape}")
print(f"Shape of train label is {y_train.shape}")
print(f"Shape of test labels is {y_test.shape}")

Shape of train set is (867000, 65)
Shape of test set is (46000, 65)
Shape of train label is (867000, 1)
Shape of test labels is (46000, 1)


In [19]:
type(y_train),type(X_train)

(numpy.ndarray, numpy.ndarray)

In [20]:
y_train[:5]

array([[13],
       [11],
       [14],
       [13],
       [10]])

#### Neural Networks

In [ ]:
#### Neural Networks

In [ ]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [54]:
n_samples, n_feats = X_train.shape
n_samples, n_feats

(867000, 65)

In [64]:
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Second LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

# The output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='rmsprop',loss='mean_squared_error')
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=50,batch_size=32)

ValueError: Input 0 of layer sequential_8 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 65]

In [132]:
n_feats

65

In [133]:
model = Sequential()

In [134]:
model.add(Dense(20, input_dim=n_feats,activation="relu", name="layer1"))
model.add(Dense(40, input_dim=n_feats,activation="relu", name="layer2"))
# model.add(Dense(16, input_dim=n_feats,name="layer3"))
model.add(Dense(1, input_dim=n_feats,name="layer3"))
model.compile(loss='mean_squared_error',
             optimizer='sgd')

In [135]:
len(model.weights)

6

In [136]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 20)                1320      
_________________________________________________________________
layer2 (Dense)               (None, 40)                840       
_________________________________________________________________
layer3 (Dense)               (None, 1)                 41        
Total params: 2,201
Trainable params: 2,201
Non-trainable params: 0
_________________________________________________________________


In [137]:
model.compile(loss='mean_squared_error',
             optimizer='sgd')

In [138]:
model.fit(X_train, y_train)

867000/867000 [==============================] - 31s 36us/sample - loss: nan


In [124]:
yhat = model.predict(X_test)
print("predict without training it: ", yhat[:5])
y_test[:10]

predict without training it:  [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]


array([[21],
       [12],
       [18],
       [15],
       [20],
       [19],
       [22],
       [19],
       [ 9],
       [23]])

In [83]:
yhat.shape

(46000, 1)

In [84]:
yhat[1000],y_test[1000]

(array([52.01563], dtype=float32), array([35]))

In [59]:
model.compile(loss='mean_squared_error',
             optimizer='sgd')

In [67]:
model.fit(X_train, y_train)

867000/867000 [==============================] - 29s 33us/sample - loss: 833.5071


In [32]:
n_samples, n_feats = X_train.shape

model = Sequential() # sequence of layers

hidden_units = 300
n_classes = 10

hidden_layer = Dense(units=hidden_units,
                input_dim=n_feats,
                kernel_initializer='constant',
                activation='sigmoid')

output_layer = Dense(units=n_classes,
                input_dim=hidden_units,
                kernel_initializer='uniform',
                activation='softmax')

model.add(hidden_layer)
model.add(output_layer)

In [ ]:
input_dim = X_train[1]
output_dim = trainY.shape[1] # one-hot label
# print predefined parameters of current model:
model = Sequential()
# applying a LSTM layer with x dim output and y dim input. Use dropout parameter to avoid overfitting
model.add(LSTM(output_dim=self.output_dim,
                       input_dim=input_dim,
                       activation=self.activation_lstm,
                       dropout_U=self.drop_out,
                       return_sequences=True))

model.add(LSTM(output_dim=self.output_dim,
                input_dim=self.output_dim,
                activation=self.activation_lstm,
                dropout_U=self.drop_out,
                return_sequences=True))
# argument return_sequences should be false in last lstm layer to avoid input dimension incompatibility with dense layer
model.add(LSTM(output_dim=self.output_dim,
                input_dim=self.output_dim,
                activation=self.activation_lstm,
                dropout_U=self.drop_out))
model.add(Dense(output_dim=self.output_dim,
                        activation=self.activation_last))
model.add(Dense(output_dim=output_dim,
                input_dim=self.output_dim,
                activation=self.activation_last))
# configure the learning process
model.compile(loss=self.loss, optimizer=self.optimizer, metrics=['accuracy'])
# train the model with fixed number of epoches
model.fit(x=trainX, y=trainY, nb_epoch=self.nb_epoch, batch_size=self.batch_size, validation_data=(testX, testY))


In [93]:
type(X_train)

numpy.ndarray